In [1]:
#Please make sure you have SparkNLP 2.4.1 and SparkNLP Enterprise 2.4.1

In [2]:
import sys, time

In [4]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader
import pyspark.sql.functions as F
#from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import StringIndexerModel
from pyspark.ml.classification import OneVsRestModel

In [5]:
concepts = concepts = spark.read.format("csv").option("header","true").load("../../../data/resolution/snomed_sample.csv")\
.withColumn("term", F.expr("lower(term)"))

In [6]:
tokenizer_chars = ["'",",","/"," ",".","|","@","#","%","&","$","[","]","(",")","-",";","="]

In [7]:
docAssembler = DocumentAssembler().setInputCol("term").setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")\
    .setSplitChars(tokenizer_chars)

pipelineModel = Pipeline().setStages([docAssembler, tokenizer]).fit(concepts)

In [8]:
embeddingsModel = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols("document", "token")\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [9]:
doc2Chunk = Doc2Chunk().setInputCols("document").setOutputCol("chunk")

chunkEmbeddings = ChunkEmbeddings()\
    .setInputCols("chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

pipelineChunkEmbeddings = PipelineModel([doc2Chunk, chunkEmbeddings])

In [10]:
concepts_embedded = PipelineModel([pipelineModel, embeddingsModel, pipelineChunkEmbeddings]).transform(concepts)

In [11]:
concepts_embedded.write.mode("overwrite").save("data/concepts_embedded")

In [12]:
concepts_embedded = spark.read.load("data/concepts_embedded")

In [13]:
#Let's check embeddings coverage
concepts_embedded.selectExpr("conceptId","explode(embeddings) as embs")\
.selectExpr("conceptId","case when embs.metadata.isOOV=='false' then 1 else 0 end as coverage")\
.groupby("conceptId").agg(F.expr("avg(coverage) as cov")).orderBy("cov").toPandas()["cov"].mean()

0.9727644122164317

In [14]:
#word_distribution = concepts_embedded.selectExpr("explode(token.result) as word").groupby("word").count()
#word_distribution.orderBy("count",ascending=True).show(100, False)

In [15]:
#word_distribution.count()

In [16]:
#SNOMED Resolution
snomed_resolver_l1 = DocumentLogRegClassifierModel.pretrained("resolve_snomed_clinical_l1", "en", "clinical/models")\
    .setInputCols("token").setOutputCol("partition")
snomed_resolver_l2 = ResourceDownloader.downloadPipeline("resolve_snomed_clinical_l2", "en", "clinical/models")
snomed_resolver_l2.stages[-1].setInputCols("partition","token","chunk_embeddings")

resolve_snomed_clinical_l1 download started this may take some time.
Approximate size to download 15.3 MB
[OK!]
resolve_snomed_clinical_l2 download started this may take some time.
Approx size to download 583.4 MB
[OK!]


ChunkEntityResolverSelector_d41a7a88595b

In [17]:
snomed_resolution = PipelineModel([snomed_resolver_l1, RecursivePipelineModel(snomed_resolver_l2)])

In [18]:
start = time.time()
transformed_full = snomed_resolution.transform(concepts_embedded)

In [19]:
predicted = transformed_full.withColumn("prediction", F.expr("partition.result[0]")).cache()
metrics = predicted.withColumn("ok",F.expr("case when prediction==topTerm then 1 else 0 end"))\
                                   .groupby("topTerm").agg(F.expr("avg(ok) as recall"), F.expr("count(ok) as tr_cnt"))\
                                    .join(
predicted.withColumn("ok",F.expr("case when prediction==topTerm then 1 else 0 end"))\
                                   .groupby("prediction").agg(F.expr("avg(ok) as precision")),F.col("topTerm")==F.col("prediction")
).withColumn("f1", F.expr("2*precision*recall/(precision+recall)")).orderBy("f1")\
.selectExpr("topTerm","tr_cnt","round(precision,3) as train_precision","round(recall,3) as train_recall","round(f1, 3) as train_f1")

In [ ]:
metrics.show(100, False)

In [ ]:
with_alternatives = predicted\
    .withColumn("resolution",F.expr("split(substring(snomed_code.metadata[0]['all_k_results'],2,length(snomed_code.metadata[0]['all_k_results'])-2),'\\\\],\\\\[')"))

In [ ]:
evaled = with_alternatives\
    .withColumn("good", F.expr("case when conceptId=snomed_code.result[0] then 1 else 0 end"))\
    .withColumn("hat5", F.expr("case when array_contains(slice(resolution, 1, 5), conceptId) then 1 else 0 end"))\
    .withColumn("hat10", F.expr("case when array_contains(slice(resolution, 1, 10), conceptId) then 1 else 0 end"))\
    .withColumn("hat20", F.expr("case when array_contains(slice(resolution, 1, 20), conceptId) then 1 else 0 end"))\
    .withColumn("hat30", F.expr("case when array_contains(slice(resolution, 1, 30), conceptId) then 1 else 0 end"))\
    .withColumn("hat500", F.expr("case when array_contains(slice(resolution, 1, 500), conceptId) then 1 else 0 end"))

In [ ]:
evaled.groupby("topTerm").agg(
    F.mean("good"), 
    F.mean("hat5"), 
    F.mean("hat10"), 
    F.mean("hat20"), 
    F.mean("hat30"), 
    F.mean("hat500"), 
    F.count("good")).orderBy("count(good)", ascending=False)\
.selectExpr("topTerm",
            "round(`avg(good)`, 2) as good",
            "round(`avg(hat5)`, 2) as hat5",
            "round(`avg(hat10)`, 2) as hat10",
            "round(`avg(hat20)`, 2) as hat20",
            "round(`avg(hat30)`, 2) as hat30",
            "round(`avg(hat500)`, 2) as hat500",
            "`count(good)` as total")\
.show(100,False)

In [ ]:
print(round((time.time()-start)/60, 2), "minutes")